In [16]:
import csv

# Project data
projects_data = [
    {"Project": 1, "Revenue": 15, "Days": 51},
    {"Project": 2, "Revenue": 20, "Days": 60},
    {"Project": 3, "Revenue": 5, "Days": 40},
    {"Project": 4, "Revenue": 25, "Days": 62},
    {"Project": 5, "Revenue": 22, "Days": 63},
    {"Project": 6, "Revenue": 17, "Days": 10}
]

# CSV file name
csv_filename = 'projects.csv'

# Create the CSV file
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=["Project", "Revenue", "Days"])
    writer.writeheader()
    for project in projects_data:
        writer.writerow(project)

print(f"File {csv_filename} created with success.")



File projects.csv created with success.


In [24]:
import pandas as pd
import numpy as np

# Class to represent each project with its id, revenue, and required researcher days
class Project:
    def __init__(self, id, revenue, days):
        self.id = id
        self.revenue = revenue
        self.days = days

# Function to read project data from a CSV file and create a list of Project objects
def read_projects_from_csv(filename):
    df = pd.read_csv(filename)
    projects = [Project(row['Project'], row['Revenue'], row['Days']) for index, row in df.iterrows()]
    return projects

# Class to represent a node in the decision tree for the branch and bound algorithm
class Node:
    def __init__(self, level, profit, bound, weight, items):
        self.level = level  # Level of the node in the tree
        self.profit = profit  # Current profit at this node
        self.bound = bound  # Upper bound of maximum possible profit from this node
        self.weight = weight  # Total weight (researcher days) used up to this node
        self.items = items  # List of project IDs included in this node

# Function to calculate an upper bound on the maximum profit achievable from the given node
def bound(node, n, W, projects):
    if node.weight >= W:
        return 0  # If the current weight exceeds the limit, bound is zero

    profit_bound = node.profit  # Initialize profit bound to current profit
    j = node.level + 1  # Start from the next level
    totweight = node.weight  # Initialize total weight to current weight

    # Add projects while total weight is within the limit
    while (j < n) and (totweight + projects[j].days <= W):
        totweight += projects[j].days
        profit_bound += projects[j].revenue
        j += 1

    # If there are more projects left, add fractional part of the next project to the bound
    if j < n:
        profit_bound += (W - totweight) * projects[j].revenue / projects[j].days

    return profit_bound

# Function to solve the knapsack problem using the branch and bound algorithm
def branch_and_bound(projects, max_days):
    n = len(projects)  # Number of projects
    # Sort projects by their revenue to days ratio in descending order
    projects.sort(key=lambda x: x.revenue / x.days, reverse=True)
    
    Q = []  # Initialize the queue for storing nodes
    v = Node(-1, 0, 0, 0, [])  # Create the initial dummy node
    maxProfit = 0  # Initialize the maximum profit
    best_items = []  # List to store the best solution

    # Calculate the bound for the initial node and add it to the queue
    v.bound = bound(v, n, max_days, projects)
    Q.append(v)

    # Branch and bound algorithm using a queue
    while Q:
        v = Q.pop(0)  # Get the next node from the queue
        if v.level == -1:
            u_level = 0  # If it's the initial node, start from the first project
        if v.level == n-1:
            continue  # If it's the last project, continue to the next iteration
        u_level = v.level + 1  # Move to the next level

        # Create the next node including the current project
        u = Node(u_level, v.profit + projects[u_level].revenue, 0, v.weight + projects[u_level].days, v.items + [projects[u_level].id])

        # Update the maximum profit and best solution if the new node is feasible and better
        if u.weight <= max_days and u.profit > maxProfit:
            maxProfit = u.profit
            best_items = u.items
        
        u.bound = bound(u, n, max_days, projects)  # Calculate the bound for the new node

        # If the bound is better than the current maximum profit, add the node to the queue
        if u.bound > maxProfit:
            Q.append(u)

        # Create the next node excluding the current project
        u = Node(u_level, v.profit, 0, v.weight, v.items)
        u.bound = bound(u, n, max_days, projects)  # Calculate the bound for this new node

        # If the bound is better than the current maximum profit, add the node to the queue
        if u.bound > maxProfit:
            Q.append(u)
    
    return maxProfit, best_items  # Return the maximum profit and the best solution

if __name__ == "__main__":
    csv_filename = 'projects.csv'  # Define the CSV file name
    max_days = 200  # Define the maximum number of researcher days available
    projects = read_projects_from_csv(csv_filename)  # Read the project data from the CSV file
    max_profit, best_solution = branch_and_bound(projects, max_days)  # Run the branch and bound algorithm

    # Print the results
    print(f"Maximum Profit: {max_profit}")
    print(f"Best Solution: {best_solution}")


Maximum Profit: 84
Best Solution: [6, 4, 5, 2]
